# Problem

Interpret a toy model trained to sort fixed-length lists.

# Setup
(No need to read)

In [1]:
!git clone https://github.com/ckkissane/mech-interp-practice.git

Cloning into 'mech-interp-practice'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 323 (delta 82), reused 32 (delta 14), pack-reused 181
Receiving objects: 100% (323/323), 38.61 MiB | 7.87 MiB/s, done.
Resolving deltas: 100% (161/161), done.


In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    # Needed for PySvelte to work, v3 came out and broke things...
    %pip install typeguard==2.13.3
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-6q8x58rh
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-6q8x58rh
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 54d548de4995a1ecc5b01b9c03aceaf0966c0eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 113.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━

In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [4]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
from dataclasses import dataclass
import datasets
from IPython.display import HTML

In [5]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

Plotting helper functions:

In [6]:
import plotly.graph_objects as go

update_layout_set = {"xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth"}
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0,labels={"x":xaxis, "y":yaxis}, **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label

    fig.show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(y=utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def lines(lines_list, x=None, mode='lines', labels=None, xaxis='', yaxis='', title = '', log_y=False, hover=None, **kwargs):
    # Helper function to plot multiple lines
    if type(lines_list)==torch.Tensor:
        lines_list = [lines_list[i] for i in range(lines_list.shape[0])]
    if x is None:
        x=np.arange(len(lines_list[0]))
    fig = go.Figure(layout={'title':title})
    fig.update_xaxes(title=xaxis)
    fig.update_yaxes(title=yaxis)
    for c, line in enumerate(lines_list):
        if type(line)==torch.Tensor:
            line = utils.to_numpy(line)
        if labels is not None:
            label = labels[c]
        else:
            label = c
        fig.add_trace(go.Scatter(x=x, y=line, mode=mode, name=label, hovertext=hover, **kwargs))
    if log_y:
        fig.update_layout(yaxis_type="log")
    fig.show()

def bar(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.bar(
        y=utils.to_numpy(tensor),
        labels={"x": xaxis, "y": yaxis},
        template="simple_white",
        **kwargs).show(renderer)

In [7]:
import transformer_lens.patching as patching
from transformer_lens import evals
import math

In [8]:
def visualize_attn_patterns(heads, local_tokens, local_cache, title: str = ""):
    labels = []
    patterns = []
    batch_index = 0

    for head in heads:
        if isinstance(head, tuple):
            layer, head_index = head
        else:
            layer, head_index = head // model.cfg.n_heads, head % model.cfg.n_heads
        patterns.append(local_cache["pattern", layer][batch_index, head_index])
        labels.append(f"L{layer}H{head_index}")
    patterns = torch.stack(patterns, dim=-1)
    attn_viz = pysvelte.AttentionMulti(tokens=model.to_str_tokens(local_tokens[batch_index]), attention=patterns, head_labels=labels)
    display(HTML(f"<h3>{title}</h3>"))
    attn_viz.show()

# Load Model

In [9]:
torch.set_grad_enabled(False)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


The smallest model I found that could solve this task is a 1L, 1 head, attn-only transformer with no biases or layernorms.

The model has already been trained and is loaded into this notebook below.

In [11]:
LIST_LEN = 10
MAX_NUM = 50
cfg = HookedTransformerConfig(
    n_layers=1,
    d_model=128,
    d_head=128,
    n_ctx=LIST_LEN*2 + 2, # BOS 1 4 2 MID 1 2 4
    d_vocab=MAX_NUM+2, # 0, 1, ..., MAX_NUM-1, BOS, MID
    d_vocab_out=MAX_NUM,
    attn_only=True,
    normalization_type=None,
    device=device,
    seed=0
)

model = HookedTransformer(cfg)
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): Identity()
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (unembed): Unembed()
)


In [12]:
filename = "mech-interp-practice/models/sort_fixed_len_list_model.pt"
state_dict = torch.load(filename)

model.load_state_dict(state_dict, strict=False);

# Task Description

The model is given inputs of the form:


[BOS, a_1, ..., a_n, MID, b_1, ..., b_n]


Where:


* BOS (50) and MID (51) are special tokens included at the same position in every example
*[a_1, ..., a_n] is the original array (unsorted). The elements range between [0,49] inclusive
* [b_1, ..., b_n] is the sorted array, sort([a_1, ..., a_n]). Thus these also range between [0, 49] inclusive


The model is trained to predict (b_1, ..., b_n) at positions (MID, ..., b_{n-1}) respectively. Note the model is trained with a causal mask, so it can not look ahead to see the answers. Duplicate elements in the array are possible. n=10 is fixed for every training example.


Below I provide a data loader and some example tokens that you can use to start investigating the model.

In [13]:
def make_data_generator(cfg, batch_size, seed=0):
    torch.manual_seed(seed)
    MID_TOKEN = cfg.d_vocab-1
    BOS_TOKEN = cfg.d_vocab-2
    while True:
        x = torch.randint(0, cfg.d_vocab_out, (batch_size, LIST_LEN))
        sorted_x = x.sort(dim=-1).values
        mid_vec = torch.ones(batch_size).unsqueeze(-1) * MID_TOKEN
        bos_vec = torch.ones(batch_size).unsqueeze(-1) * BOS_TOKEN

        tokens = torch.cat([bos_vec, x, mid_vec, sorted_x], dim=-1).to(torch.long)
        yield tokens

batch_size = 256
train_data_loader = make_data_generator(cfg, batch_size, seed=42)
test_data = next(train_data_loader).to(device)
print(test_data.shape)
print(test_data[:5])

torch.Size([256, 22])
tensor([[50, 42, 17, 26, 14, 26, 35, 20, 24,  0, 13, 51,  0, 13, 14, 17, 20, 24,
         26, 26, 35, 42],
        [50, 28, 14, 10,  4, 31, 22, 15, 45, 17,  6, 51,  4,  6, 10, 14, 15, 17,
         22, 28, 31, 45],
        [50, 49, 26, 23, 11, 49, 13, 41, 19, 37, 19, 51, 11, 13, 19, 19, 23, 26,
         37, 41, 49, 49],
        [50, 22, 30, 25, 29, 33, 14, 39, 26, 32, 10, 51, 10, 14, 22, 25, 26, 29,
         30, 32, 33, 39],
        [50, 36, 22, 27, 19,  7, 23, 43, 44, 43, 27, 51,  7, 19, 22, 23, 27, 27,
         36, 43, 43, 44]], device='cuda:0')


The model was trained to minimize cross entropy loss. The loss function used during training is provided below:

In [14]:
def loss_fn(logits, tokens):
    logits = logits[:, -(LIST_LEN+1):-1, :]
    log_probs = logits.log_softmax(dim=-1)
    labels = tokens[:, -LIST_LEN:]
    correct_log_probs = log_probs.gather(dim=-1, index=labels[..., None])[..., 0]
    return -correct_log_probs.mean()

It can be useful to convert tokens to a list of strings for plotly / CircuitsVis labels. Here is a function that you can optionally use for this:

In [15]:
def tokens_to_plotly_labels(tokens, incl_pos=True):
    if tokens.ndim == 2:
        tokens = tokens[0]
    if incl_pos:
        labels = [f"{tok}_{i}" for i, tok in enumerate(tokens)]
    else:
        labels = list(map(str, tokens.tolist()))
    labels[0] = 'BOS'
    labels[LIST_LEN+1] = 'MID'
    return labels

print("incl_pos=True:", tokens_to_plotly_labels(test_data[0]))
print("incl_pos=False:", tokens_to_plotly_labels(test_data[0], incl_pos=False))

incl_pos=True: ['BOS', '42_1', '17_2', '26_3', '14_4', '26_5', '35_6', '20_7', '24_8', '0_9', '13_10', 'MID', '0_12', '13_13', '14_14', '17_15', '20_16', '24_17', '26_18', '26_19', '35_20', '42_21']
incl_pos=False: ['BOS', '42', '17', '26', '14', '26', '35', '20', '24', '0', '13', 'MID', '0', '13', '14', '17', '20', '24', '26', '26', '35', '42']
